In [ ]:
from corda import CORDA
import pandas as pd
import numpy as np
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model, load_model
import os

In [ ]:
os.environ['GUROBI_HOME'] = "/home/jiang817/cybergut_project/Software/gurobi_license"
os.environ['GRB_LICENSE_FILE'] = "/home/jiang817/cybergut_project/Software/gurobi_license/gurobi.lic"

In [ ]:
model = read_sbml_model('/home/jiang817/cybergut_project/General_models/Recon3D.xml')

In [ ]:
for i in range (len(model.reactions)):
    if model.reactions[i].id.startswith('EX_'):
        model.reactions[i].bounds = (-100, 100)

In [ ]:
for i in model.metabolites:
    print(i)

In [ ]:
rxn_score = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_iMAT_INPUT/rxn_score_latest.csv')

In [ ]:
rxn_score

In [ ]:
for idx in range(len(rxn_score)):
    if rxn_score.iloc[idx, 1] > 0:
        print(rxn_score.iloc[idx, 1])
        rxn_score.iloc[idx, 1] = 3
    if rxn_score.iloc[idx, 1] == 0:
        print(rxn_score.iloc[idx, 1])
        rxn_score.iloc[idx, 1] = -1

In [ ]:
for i in rxn_score['0']:
    print(i)

In [ ]:
rxn_score.fillna(0, inplace=True)

In [ ]:
confidence = {}
for i in range(len(rxn_score)):
    key = rxn_score.iloc[i, 0]
    value = rxn_score.iloc[i, 1]
    print(key, value)
    confidence[key] = value

In [ ]:
Met = pd.read_csv('//home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_CORDA_INPUT/KeyMets_Recon3D_XML.txt', header=0)

In [ ]:
Met

In [ ]:
Met = Met['Met'].to_list()

In [ ]:
Met

In [ ]:
ts_model = CORDA(model, confidence, Met)

In [ ]:
ts_model.build()

In [ ]:
print(ts_model)

In [ ]:
num = 0
for i in ts_model.included:
    if ts_model.included[i] == True:
        num += 1
print(num)

In [ ]:
for k in ts_model.included.items():
    print(k)

In [ ]:
import cobra

In [ ]:
model.remove_reactions

In [ ]:
for i in ts_model.included:
    if ts_model.included[i] == False:
        model.remove_reactions(i, remove_orphans=True)
        

In [ ]:
write_sbml_model(model, '/home/jiang817/cybergut_project/Draft_model/Draft_model_with_CORDA/draft_model_CORDA_BIGG.xml')

In [ ]:
model